In [96]:
import arcpy;
import os,sys;
import requests,csv;
import datetime;
from dateutil.rrule import rrule,DAILY,YEARLY;
from time import sleep;

start_date  = datetime.date(1990,1,1);
end_date    = datetime.date(2020,12,31);

results_fgdb = os.getcwd() + os.sep + '..'+ os.sep + 'results.gdb';
target_dir   = os.getcwd() + os.sep + 'prism';

if not os.path.exists(target_dir):
    os.mkdir(target_dir);

stations = results_fgdb  + os.sep + 'SWC_Station_Universe';
stations_cnt = arcpy.GetCount_management(stations)[0];
print("  Initial SWC Station Universe Count: " + str(stations_cnt));


  Initial SWC Station Universe Count: 5144


In [97]:
%%time

stations   = results_fgdb + os.sep + 'SWC_Station_Universe';
tickers = {};

flds = [
     'StationId'
    ,'SHAPE@'
];

ticker_cnt  = 1;
ticker_size = 5;
ticker_max  = None;

print("  creating station tickers as a hash.");
with arcpy.da.SearchCursor(
     in_table     = stations
    ,field_names  = flds
    ,where_clause = "CONUSFlag = 'Y'"
) as incur:

    ticker = 1;
    outfile = None;
    for row in incur:
    
        point = row[1].firstPoint;
        lat  = str(round(point.Y,8));
        lon  = str(round(point.X,8));
        name = row[0];
        
        if ticker == 1:
            tickers[ticker_cnt] = {
                 "lats" : lat
                ,"lons" : lon
                ,"names": name
            };
            
        else:
            tickers[ticker_cnt]['lats']  = tickers[ticker_cnt]['lats']  + '|' + lat;
            tickers[ticker_cnt]['lons']  = tickers[ticker_cnt]['lons']  + '|' + lon;
            tickers[ticker_cnt]['names'] = tickers[ticker_cnt]['names'] + '|' + name;

        ticker += 1;
        if ticker == ticker_size + 1:
            ticker_cnt += 1;
            ticker = 1;
            
            if ticker_max is not None and ticker_cnt > ticker_max:
                break;
            
#tickers
            

  creating station tickers as a hash.
Wall time: 230 ms


In [101]:
sleeper = 1;
url = 'https://prism.oregonstate.edu/explorer/dataexplorer/rpc.php';
pickup = 'https://prism.oregonstate.edu/explorer/tmp/'
# These headers are probably utterly irrelevant to the prism servers
headers = {
     'Host'            : 'prism.oregonstate.edu' 
    ,'Connection'      : 'keep-alive'
    ,'Content-Length'  : '7434'
    ,'Pragma'          : 'no-cache'
    ,'Cache-Control'   : 'no-cache'
    ,'sec-ch-ua'       : '"Chromium";v="92", " Not A;Brand";v="99", "Google Chrome";v="92"'
    ,'sec-ch-ua-mobile': '?0'
    ,'Sec-Fetch-Dest'  : 'empty'
    ,'Sec-Fetch-Mode'  : 'cors'
    ,'Sec-Fetch-Site'  : 'same-origin'
    ,'Accept'          : 'application/json, text/javascript, */*; q=0.01'
    ,'Content-Type'    : 'application/x-www-form-urlencoded; charset=UTF-8'
    ,'Accept-Language' : 'en-us'
    ,'Accept-Encoding' : 'gzip, deflate'
    ,'Origin'          : 'https://www.prism.oregonstate.edu'
    ,'Referer'         : 'https://www.prism.oregonstate.edu/explorer/bulk.php'
    ,'User-Agent'      : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36'
    ,'X-Requested-With': 'XMLHttpRequest'  
    ,'DNT'             : '1'
}

for k,v in tickers.items():
    
    # Use this logic bypass if you need to restart following a network failure
    if k >= 0:
        
        print(" Processing " + str(k));

        target_file = target_dir + os.sep + 'batch' + str(k) + '.csv';
        if os.path.exists(target_file):
            os.remove(target_file);

        for yr in rrule(
             freq    = YEARLY
            ,dtstart = start_date
            ,until   = end_date
        ):
            payload_outer = {
                 'call'     : 'pp/daily_timeseries_mp'
                ,'proc'     : 'gridserv'
                ,'lons'     : v['lons']
                ,'lats'     : v['lats']
                ,'names'    : v['names']
                ,'spares'   : '4km'
                ,'interp'   : 0
                ,'stats'    : 'tmin tmax'
                ,'units'    : 'eng'
                ,'range'    : 'daily'
                ,'start'    : yr.strftime("%Y") + '0101'
                ,'end'      : yr.strftime("%Y") + '1231'
                ,'stability': 'stable'
            }

            r_outer = requests.post(
                 url
                ,data = payload_outer
                ,headers = headers
            );
            resp_outer = r_outer.json();

            result_url = None;
            for waiting in range(35):
                payload_inner = {
                     'call' :'pp/checkup'
                    ,'proc' : 'gridserv'
                    ,'gricket': resp_outer['gricket']
                }

                r_inner = requests.post(
                     url
                    ,data = payload_inner
                    ,headers = headers
                );
                resp_inner = r_inner.json();

                if 'delay' in resp_inner:
                    #print(resp_inner['delay']['status']);
                    pass
                elif 'result' in resp_inner:
                    result_url = resp_inner['result']['csv'];
                    break;

                sleep(sleeper);

            if result_url is None:
                raise Exception('never got back results for ' + str(k) + ' for ' + yr.strftime("%Y"));

            with open(target_file,'ab') as f:

                with requests.get(
                     pickup + result_url
                    ,stream = True
                ) as r:
                    skip = 1;
                    for line in r.iter_lines():
                        if skip > 11 and len(line) > 0:
                            f.write(line+'\n'.encode());
                        skip += 1;
                    

 Processing 718
 Processing 719
 Processing 720
 Processing 721
 Processing 722
 Processing 723
 Processing 724
 Processing 725
 Processing 726
 Processing 727
 Processing 728
 Processing 729
 Processing 730
 Processing 731
 Processing 732
 Processing 733
 Processing 734
 Processing 735
 Processing 736
 Processing 737
 Processing 738
 Processing 739
 Processing 740
 Processing 741
 Processing 742
 Processing 743
 Processing 744
 Processing 745
 Processing 746
 Processing 747
 Processing 748
 Processing 749
 Processing 750
 Processing 751
 Processing 752
 Processing 753
 Processing 754
 Processing 755
 Processing 756
 Processing 757
 Processing 758
 Processing 759
 Processing 760
 Processing 761
 Processing 762
 Processing 763
 Processing 764
 Processing 765
 Processing 766
 Processing 767
 Processing 768
 Processing 769
 Processing 770
 Processing 771
 Processing 772
 Processing 773
 Processing 774
 Processing 775
 Processing 776
 Processing 777
 Processing 778
 Processing 779
 Process